In [4]:
import json
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textstat import syllable_count, flesch_reading_ease, automated_readability_index
from datetime import datetime

In [5]:
with open('C:/Hammad Aslam/BS IT (post ADP)/3rd Semester/Capstone Project/Project/backend/datasets/categories/allFiles/computers_laptops.json', 'r') as f:
    data = json.load(f)
len(data[0]['reviews'])

5

In [6]:
for product in data:
    array = []
    for review in product['reviews']:
        array.append(review['review_helpfulness'])
        
for product in data:
    for review in product['reviews']:
        sia = SentimentIntensityAnalyzer()
        sentiment_scores = sia.polarity_scores(review['review_body'])
        review['sentiment'] = "Positive" if sentiment_scores['compound'] >= 0.5 else "Negative"
        # Preprocessing
        text = review['review_body']
        tokens = nltk.word_tokenize(text)
        tokens = [t for t in tokens if t.isalpha() and t.lower() not in nltk.corpus.stopwords.words('english')]
        text = ' '.join(tokens)
        # Text Metrics Extraction
        word_count = len(tokens)
        sentence_count = len(nltk.sent_tokenize(text))
        # syllable_count = Pyphen().inserted(text).count('-') + 1
        character_count = len(text.replace(' ', ''))
        fres_score = flesch_reading_ease(text)
        ari_score = automated_readability_index(text)
        # print(sentiment_scores['compound'])
        date_str = review['reviewer_country_date'] if review['reviewer_country_date'] != "" else "January 01, 2010"
        review_timestamp = datetime.strptime(date_str, "%B %d, %Y")

        # Get the current time (corrected line)
        current_time = datetime.now()

        # Calculate the time difference
        time_difference = current_time - review_timestamp

        # Print the difference in a human-readable format (optional)
        # print(f"The time difference is {time_difference}")

        # Get the difference in specific units (days, seconds, etc.)
        days_difference = time_difference.days
        decay_factor = 1 - (0.0001 * days_difference)
        # print(f"decay_factor: {decay_factor}")
        rating = float(review['review_rating'])
        # helpfulness_score = (rating * 0.2) + (character_count * 0.1) + (word_count * 0.1) + (sentence_count * 0.1) + (100 - fres_score) * 0.2 + (ari_score * 0.2)
        # helpfulness_score = ((rating * 0.2) + (character_count * 0.02) + (
        #     word_count * 0.13) + (sentence_count * 0.2) + ((100 - fres_score) * 0.1) + (ari_score * 0.1) + (sentiment_scores['compound'] * 0.25)) * decay_factor
        if int(review['review_votes']) >= 4:
            review['review_helpfulness'] = 'helpful'
        else:
            review['review_helpfulness'] = 'unhelpful'
        # print((review['review_helpfulness'] - min(array)) / (max(array) - min(array)))

In [7]:
with open('C:/Hammad Aslam/BS IT (post ADP)/3rd Semester/Capstone Project/Project/backend/datasets/categories/allFiles/computers_laptops.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)